In [1]:
import requests
import os
import gzip
import glob

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

from slugify import slugify

In [2]:
YEARS = [2015, 2016]
START_AMOUNT = 420000  # PLN, roughly 100k EUR
MAX_DIV = 300
#  Both ranges are non-inclusive!
RANGES = [START_AMOUNT // i for i in range(1, MAX_DIV)] + [i for i in range(START_AMOUNT // MAX_DIV, 0, -100)] + [0]
len(RANGES)

314

In [3]:
# These break the export, probably because of non-windows-1250 characters
# The downloader queries around these sums and so these recipients are missing from the years
SPECIALS = {
    2015: [
        21016.97, # http://beneficjenciwpr.minrol.gov.pl/outrecords/view/12485559
        51755.63, # http://beneficjenciwpr.minrol.gov.pl/outrecords/view/13334673
    ],
    2016: [
        18805.66, # http://beneficjenciwpr.minrol.gov.pl/outrecords/view/16350294
    ]
}

In [4]:
BASE_URL = 'http://beneficjenciwpr.minrol.gov.pl/search/export/csv/'

In [5]:
params = {'year': 2015, 'totalfrom': 420000, 'totalto': None}

def get_url(params):
    return BASE_URL + '/'.join('{k}:{v}'.format(k=k, v=str(v).replace(".", ","))
                               for k, v in params.items() if v is not None) + '/sort:total/direction:asc'


In [6]:
def download(year, amount, total_to):
    url = get_url({'year': year, 'totalfrom': amount, 'totalto': total_to})
    filename = 'data/{year}_{amount}_{total_to}.csv.gz'.format(year=year, amount=amount, total_to=total_to)
    if os.path.exists(filename):
        return
    print(filename, url)
    response = requests.get(url, stream=True)
    with gzip.open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024): 
            if chunk:
                f.write(chunk)

                
for year in YEARS:
    specials = SPECIALS.get(year, [])
    total_to = None
    for amount in RANGES:
        specs = []
        if total_to is not None:
            specs = [s for s in specials if amount < s < total_to]
        if specs:
            download(year, amount, specs[0])
            download(year, specs[0], total_to)
        else:
            download(year, amount, total_to)
        total_to = amount+0.001


In [7]:
!rm data/2016_18260_19090.001.csv.gz

rm: data/2016_18260_19090.001.csv.gz: No such file or directory


In [8]:
def get_year(year):
    for filename in tqdm_notebook(glob.glob('data/{}_*.csv.gz'.format(year))):
#         print(filename)
        yield pd.read_csv(filename, compression='gzip', encoding='windows-1250', sep=';')

In [9]:
# with gzip.open('data/2016_11666_12000.001.csv.gz') as f:
#     print(f.read()[:10400].decode('windows-1250'))

In [14]:
def apply_fixes(df):
    df = df.rename(columns={
        'Imię': 'recipient_firstname',
        'Nazwisko': 'recipient_lastname',
        'Nazwa': 'recipient_name',
        'Gmina': 'recipient_location',
        'Kod pocztowy': 'recipient_postcode',
        'Rok': 'year'
    })
    df = df.drop(columns=['Suma'])
    df['recipient_firstname'] = df['recipient_firstname'].fillna('').apply(str)
    df['recipient_lastname'] = df['recipient_lastname'].fillna('').apply(str)
    df['recipient_name'] = df['recipient_name'].where(df['recipient_name'].notnull(),
                                                      df['recipient_firstname'] + ' ' + df['recipient_lastname'])
    df = df.drop(columns=['recipient_firstname', 'recipient_lastname'])
    df['recipient_id'] = df.apply(lambda x: '%s-%s-%s' % (slugify(x['recipient_name']), x['recipient_postcode'], x['year']), 1)
    return df

In [15]:
df = pd.concat(get_year(2016))
df = apply_fixes(df)
df.head()

HBox(children=(IntProgress(value=0, max=315), HTML(value='')))

/Users/sw/.virtualenvs/notebook36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
/Users/sw/.virtualenvs/notebook36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


,year,recipient_name,recipient_location,recipient_postcode,I.1 System płatności jednolitej tytuł III,"I.2 System jednolitej płatności obszarowej Tytuł V, rozdział 2","I.3 Wsparcie dla wytwórców buraków cukrowych i trzciny cukrowej - Tytuł IV, rozdział 1, sekcja 7","I.4 Płatności z tytułu wołowiny i cielęciny - Tytuł IV, rozdział 1 sekcja 11","I.5 Premie z tytułu mięsa baraniego i koziego - Tytuł IV, rozdział 1, sekcja 10","I.6 Płatności specjalne z tytułu bawełny - Tytuł IV, rozdział 1 sekcja 6",...,V/B.4.3 Jakość życia/różnicowanie,V/B.4.4 b) Współpraca transnarodowa i międzyregionalna,"V/B.4.5 c) Prowadzenie lokalnej grupy działania, nabywanie umiejętności i prowadzenie na terytorium aktywizacji",V/B.5 Art. 66 Pomoc techniczna,V/B.6 Dodatkowe płatności bezpośrednie w Bułgarii i Rumunii,VI/A.1 Działania informacyjne i promocyjne zapewniane w ramach rozporządzenia (WE) nr 1144/2014,VI/B.1 Działania informacyjne i promocyjne zapewniane w rozporządzeniu (WE) nr 3/2008,VI/C.1 POSEI,VI/D.1 Mniejsze wyspy Morza Egejskiego,recipient_id
0,2016,Małe gospodarstwo 100468,KRZCZONÓW,23-110,"0,00","0,00","0,00","0,00","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",male-gospodarstwo-100468-23-110-2016
1,2016,Małe gospodarstwo 100598,JAKUBÓW,05-306,"0,00","0,00","0,00","0,00","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",male-gospodarstwo-100598-05-306-2016
2,2016,Małe gospodarstwo 101861,DĘBNO,32-852,"0,00","0,00","0,00","0,00","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",male-gospodarstwo-101861-32-852-2016
3,2016,Małe gospodarstwo 10236,MAJDAN KRÓLEWSKI,36-110,"0,00","0,00","0,00","0,00","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",male-gospodarstwo-10236-36-110-2016
4,2016,Małe gospodarstwo 102457,HERBY,42-284,"0,00","0,00","0,00","0,00","0,00","0,00",...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",male-gospodarstwo-102457-42-284-2016


In [16]:
len(df)

1403521

In [17]:
cols = 'recipient_name|recipient_location|recipient_id|recipient_postcode|year'.split('|')
scheme_cols = list(set(df.columns) - set(cols))

In [18]:
len(scheme_cols)

101

In [19]:
# df['Suma'] = pd.to_numeric(df['Suma'].str.replace(',', '.'))
for c in scheme_cols:
    df[c] = pd.to_numeric(df[c].str.replace(',', '.'))

In [20]:
# mismatch = np.isclose(df[scheme_cols].sum(1), df['Suma'] , atol=1)
# len(df) - mismatch.sum()

In [21]:
df_final = pd.melt(df, id_vars=cols,  var_name='scheme', value_name='amount', value_vars=scheme_cols)
df_final.head()

,recipient_name,recipient_location,recipient_id,recipient_postcode,year,scheme,amount
0,Małe gospodarstwo 100468,KRZCZONÓW,male-gospodarstwo-100468-23-110-2016,23-110,2016,V/B.1.14 Art. 33 Wsparcie ukierunkowane na kon...,0.0
1,Małe gospodarstwo 100598,JAKUBÓW,male-gospodarstwo-100598-05-306-2016,05-306,2016,V/B.1.14 Art. 33 Wsparcie ukierunkowane na kon...,0.0
2,Małe gospodarstwo 101861,DĘBNO,male-gospodarstwo-101861-32-852-2016,32-852,2016,V/B.1.14 Art. 33 Wsparcie ukierunkowane na kon...,0.0
3,Małe gospodarstwo 10236,MAJDAN KRÓLEWSKI,male-gospodarstwo-10236-36-110-2016,36-110,2016,V/B.1.14 Art. 33 Wsparcie ukierunkowane na kon...,0.0
4,Małe gospodarstwo 102457,HERBY,male-gospodarstwo-102457-42-284-2016,42-284,2016,V/B.1.14 Art. 33 Wsparcie ukierunkowane na kon...,0.0


In [22]:
df_final = df_final[(df_final['amount'] != 0.0) & df_final['amount'].notnull()]
df_final.head()

,recipient_name,recipient_location,recipient_id,recipient_postcode,year,scheme,amount
488407,CUKIERNICZA SPÓŁDZIELNIA ROKSANA,STRZYŻÓW,cukiernicza-spoldzielnia-roksana-38-100-2016,38-100,2016,V/B.1.14 Art. 33 Wsparcie ukierunkowane na kon...,13385.05
1086556,SPÓŁDZIELNIA PSZCZELARSKA APIS W LUBLINIE,LUBLIN,spoldzielnia-pszczelarska-apis-w-lublinie-20-4...,20-471,2016,V/B.1.14 Art. 33 Wsparcie ukierunkowane na kon...,11058.46
1124260,WYTWÓRCZA SPÓŁDZIELNIA PRACY SPOŁEM,KIELCE,wytworcza-spoldzielnia-pracy-spolem-25-726-2016,25-726,2016,V/B.1.14 Art. 33 Wsparcie ukierunkowane na kon...,1445353.00
2808974,JADWIGA MAKÓWKA,GRODZISKO DOLNE,jadwiga-makowka-37-306-2016,37-306,2016,II.7 Dobrowolne wsparcie związane z produkcją ...,166.08
2808977,KATARZYNA NOWAK,NOWY KORCZYN,katarzyna-nowak-28-136-2016,28-136,2016,II.7 Dobrowolne wsparcie związane z produkcją ...,352.93


In [23]:
df_final.to_csv('pl_2016.csv.gz', compression='gzip', index=False)